## Benchmarking quantum circuits for bad qubits and edges

We identify bad qubits and edges for different hardware. We use the calibration data for this purpose. The bad qubits are determined by their SPAM probabilities only. It may be possible to put in 1q gate error probabilities as well. But usually they are order of magnitudes lesser, and hence can be ignored. The bad edges are determined by the 2q gate error probabilities.

We shall use the statistical method of `z-score` to identify outliers.

In [1]:
import numpy as np

def identify_bad_qubits(backend, z_score:float=1.0):
    """
        If some qubits are really bad, and statistical outliers, it is useful
        to remove those qubits as long as the isomorphism of the mapped circuit
        is not lost
    """
    spam_error = {}
    
    for qubit in range(backend.configuration().n_qubits):
        spam_error[qubit] = backend.properties().readout_error(qubit)

    retained = {}
    outliers = {}

    mean_score = np.mean(list(spam_error.values()))
    std_score = np.std(list(spam_error.values()))

    for qubit, score in spam_error.items():
        if (score-mean_score)/std_score > z_score:
            outliers[qubit] = score
        else:
            retained[qubit] = score

    return outliers, retained, np.median(list(spam_error.values()))

In [2]:
def identify_bad_edges(backend, z_score:float=1.0):
    """
        If some edges are really bad, and statistical outliers, it is useful
        to remove those edges as long as the isomorphism of the mapped circuit
        is not lost
    """
    if 'cx' in backend.configuration().basis_gates:
        basis_2q_gate = 'cx'
    elif 'ecr' in backend.configuration().basis_gates:
        basis_2q_gate = 'ecr'
    else:
        basis_2q_gate = 'cz'

    gate_2q_error = {}

    for qubit1 in range(backend.configuration().n_qubits):
        for qubit2 in range(backend.configuration().n_qubits):
            if [qubit1, qubit2] in backend.configuration().coupling_map:
                gate_2q_error[(qubit1, qubit2)] = backend.properties().gate_error(basis_2q_gate, [qubit1, qubit2])

    retained = {}
    outliers = {}

    mean_score = np.mean(list(gate_2q_error.values()))
    std_score = np.std(list(gate_2q_error.values()))

    for edge, score in gate_2q_error.items():
        if std_score == 0:
            retained[edge] = score
        elif (score-mean_score)/std_score > z_score:
            outliers[edge] = score
        else:
            retained[edge] = score

    return outliers, retained, np.median(list(gate_2q_error.values()))

In [3]:
def create_punctured_coupling_map(backend, spam_outliers: dict, edge_outliers: dict) -> list:
    """
        Create punched coupling map by removing spam and edge outliers
    """
    coupling_map = backend.configuration().coupling_map
    punched_coupling_map = []

    bad_qubits = list(spam_outliers.keys())
    bad_edges = list(edge_outliers.keys())

    for edge in coupling_map:
        if edge not in bad_edges and edge[0] not in bad_qubits and edge[1] not in bad_qubits:
            punched_coupling_map.append(edge)

    return punched_coupling_map

In [25]:
def obtain_median_and_worst_qubit(backend, qubit_list: list, median: float) -> float:
    spam_error_list = []
    for qubit in qubit_list:
        spam_error_list.append(backend.properties().readout_error(qubit))

    qubit_worst = max(spam_error_list)
    bad_qubits_count = [spam for spam in spam_error_list if spam > median]

    return qubit_worst, len(bad_qubits_count)

In [26]:
def obtain_median_and_worst_edge(backend, edge_list: list, median: float) -> float:
    if 'cx' in backend.configuration().basis_gates:
        basis_2q_gate = 'cx'
    elif 'ecr' in backend.configuration().basis_gates:
        basis_2q_gate = 'ecr'
    else:
        basis_2q_gate = 'cz'
    
    edge_error_list = []
    for edge in edge_list:
        edge_error_list.append(backend.properties().gate_error(basis_2q_gate, edge))

    edge_worst = max(edge_error_list)
    bad_edge_count = [edge for edge in edge_error_list if edge > median]

    return edge_worst, len(bad_edge_count)

### Test for hardware

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService(channel='ibm_quantum', instance='client-enablement/solutions/demo-testing')

In [7]:
list_of_hardware = service.backends()
list_of_hardware

[<IBMBackend('ibm_kyiv')>,
 <IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_nazca')>,
 <IBMBackend('ibm_sherbrooke')>,
 <IBMBackend('ibm_torino')>,
 <IBMBackend('ibm_kyoto')>]

In [32]:
benchmark_data = {}
z_scores = [1.5,1.0,0.5,0.1,0.05]

In [33]:
import networkx as nx

for hardware in list_of_hardware:
    if hardware.name == 'ibm_nazca':
        continue
    benchmark_data[hardware.name] = {}

    for z_score in z_scores:
        benchmark_data[hardware.name][z_score] = {}
        benchmark_data[hardware.name][z_score]['spam'] = {}
        benchmark_data[hardware.name][z_score]['edge'] = {}
        
        # obtain spam outliers
        spam_outliers, spam_retained, qubit_median = identify_bad_qubits(hardware, z_score=z_score)
        benchmark_data[hardware.name][z_score]['spam']['outliers'] = spam_outliers

        # obtain edge outliers
        edge_outliers, edge_retained, edge_median = identify_bad_edges(hardware, z_score=z_score)
        benchmark_data[hardware.name][z_score]['edge']['outliers'] = edge_outliers

        # obtain punctured coupling map
        punctured_coupling_map = create_punctured_coupling_map(hardware, 
                                                               spam_outliers, edge_outliers)

        # find connected components
        graph = nx.Graph()
        graph.add_edges_from(punctured_coupling_map)
        connected_components = list(nx.connected_components(graph))
        component_sizes = [len(component) for component in connected_components]
        component_sizes.sort(reverse=True)
        benchmark_data[hardware.name][z_score]['component_size'] = component_sizes

        # get median and worst data
        qubit_worst, bad_qubit_count = obtain_median_and_worst_qubit(hardware, spam_retained, qubit_median)
        edge_worst, bad_edge_count = obtain_median_and_worst_edge(hardware, edge_retained, edge_median)
        
        benchmark_data[hardware.name][z_score]['spam']['median'] = qubit_median
        benchmark_data[hardware.name][z_score]['spam']['worst'] = qubit_worst
        benchmark_data[hardware.name][z_score]['spam']['num_qubit_worse_than_median'] = bad_qubit_count

        benchmark_data[hardware.name][z_score]['edge']['median'] = edge_median
        benchmark_data[hardware.name][z_score]['edge']['worst'] = edge_worst
        benchmark_data[hardware.name][z_score]['edge']['num_edge_worse_than_median'] = bad_edge_count

In [38]:
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')

filename = 'benchmark_punctured_coupling_map_'+date+'_calibration_data.pkl'

In [40]:
import pickle

file = open(filename, 'wb')
pickle.dump(benchmark_data, file)